- ## <b>Acompanhamento de precificação: Medicamentos</b>


In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
from bmldev.pdleo import cria_dfs, le_base_excel, converte_tipos_colunas, branco_para_nan
from datetime import datetime

In [3]:
dir_input = "./bases/"
dir_output = "./relatorio/"

In [4]:
dt_hoje = '20210613'
dt_ultimo_relatorio = '20210609'

- ## <b>Leitura de bases</b>

In [5]:
%%time
dfs_txt = cria_dfs(
                nomes_bases=['zbfa', 'eina','mean'],
    
                palavras_chave=['abcfarma','eina','mean'],
    
                cols=[16,4,2],
    
                multiplas_bases_txt=[False],
                
                diretorio=dir_input

    
)

Lendo base zbfa...ok
Lendo base eina...ok
Lendo base mean...ok
Wall time: 3.97 s


In [6]:
df_dadosfarma = le_base_excel(dir_input, 'banco de dados')

In [7]:
df_iqvia = le_base_excel(dir_input, 'IQVIA_Norte')

In [8]:
## Vamos utilizar para comparar o último desconto sobre PF cadastrado para cada SKU

df_desc_ant =le_base_excel(dir_input, 'desonto', 'Original')

- ## <b>Processamento</b>

- ### df_zbfa

In [137]:
df_zbfa = dfs_txt['zbfa'].copy()

In [138]:
df_zbfa = df_zbfa.iloc[:,:12]

In [139]:
df_zbfa.columns = ['ean', 'pf_18', 'pmc_18', 'pf_17', 'pmc_17', 'pf_17_5', 'pmc_17_5', 'data', 'pf_17_alc', 'pmc_17_alc', 'pf_17_5_alc', 'pmc_17_5_alc']

In [140]:
branco_para_nan(df_zbfa, sobrescreve=True)

In [141]:
zbfa_float_col = [col for col in df_zbfa.columns if [p for p in ['pf', 'pmc'] if p in col]]

In [142]:
df_zbfa[zbfa_float_col] = df_zbfa[zbfa_float_col].astype(float)

In [143]:
converte_tipos_colunas(df_zbfa, {'ean':str, 'data':'data'}, sobrescreve=True)

In [144]:
df_zbfa = df_zbfa.sort_values('data', ascending=False).drop_duplicates('ean').drop(columns='data')

- ### df_desc_ant

In [145]:
df_desc_ant =df_desc_ant[['codigo_de_barras_do_medicamento','material','desc_pf','desc_pmc']].copy()

In [146]:
df_desc_ant.columns = ['ean','material','ult_desc_pf','ult_desc_pmc']

In [147]:
converte_tipos_colunas(df_desc_ant, {'ean':str, 'material':str}, sobrescreve=True)

- ### df_eina

In [148]:
df_eina= dfs_txt['eina'].copy()

In [149]:
converte_tipos_colunas(df_eina, {'fornecedor':str, 'material':str}, sobrescreve=True)

In [150]:
df_eina = df_eina[df_eina.f == 'X']

In [151]:
df_eina = df_eina[['material', 'fornecedor']]

- ### df_mean

In [136]:
df_mean= dfs_txt['mean'].copy()

In [153]:
converte_tipos_colunas(df_mean, {'ean/upc':str, 'material':str}, sobrescreve=True)

In [154]:
df_mean.columns = ['material','ean']

- ### df_desc_ant- ### df_iqvia

In [157]:
## Mantendo tudo o que  temos de acordo com o relatório do IQVIA
df_iqvia = df_iqvia[df_iqvia['status'] !='Não tem']

In [158]:
converte_tipos_colunas(df_iqvia, {'ean':str, 'material':str}, sobrescreve=True)

- ### df_dadosfarma


In [159]:
df_dadosfarma = df_dadosfarma[['material', 'categ_loja','condicao','cod_forn_ult_x', 'nome_forn_ult_x', 'pmm', 'cmv_ult']]

In [160]:
df_dadosfarma.columns = ['material', 'categ_loja','condicao','cod_forn_ult', 'nome_forn_ult', 'pmm', 'custo_ult_entrada']

In [166]:
converte_tipos_colunas(df_dadosfarma, {'cod_forn_ult':str, 'material':str}, sobrescreve=True)

C:\Users\nathalia.tavares\Documents\Desconto sobre PF Med\bmldev\pdleo.py:259: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(tipo)


In [174]:
df_dadosfarma['cod_forn_ult']= df_dadosfarma.cod_forn_ult.str.split(pat='.',expand=True)

<ipython-input-174-7b803ff9033c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dadosfarma['cod_forn_ult']= df_dadosfarma.cod_forn_ult.str.split(pat='.',expand=True)


In [186]:
relatorio = relatorio.merge(df_zbfa[['ean','pf_18','pmc_18']], on='ean',how='left')

In [198]:
relatorio = relatorio.merge(df_desc_ant[['material','ult_desc_pf', 'ult_desc_pmc']], on='material',how='left')

In [203]:
relatorio.pf_18 = relatorio.pf_18.fillna(0)
relatorio.pmc_18 = relatorio.pmc_18.fillna(0)
relatorio.ult_desc_pf = relatorio.ult_desc_pf.fillna(0)
relatorio.ult_desc_pmc = relatorio.ult_desc_pmc.fillna(0)

In [212]:
relatorio['desconto_sobre_pf'] = relatorio.apply(lambda row: round(1-(row['custo_ult_entrada']/row['pf_18']),2) if row['pf_18'] > 0 else 0, axis=1)

In [210]:
relatorio['mark up'] = relatorio.apply(lambda row: 'X' if row['pf_18'] == 0 and row['pmc_18'] == 0 else np.nan, axis=1)

In [219]:
writer = pd.ExcelWriter(dir_output + "Desconto sobre PF Medicamentos" + 
                        pd.to_datetime(datetime.today()).strftime('%Y%m%d') +
                        ".xlsx", engine="xlsxwriter")

In [220]:
relatorio.to_excel(writer, index=False)

In [221]:
writer.save()